## Chapter 5 Monte Carlo Methods

In contrast to Chapter 4, we don't assume to have complete knowledge of the environment here. Monte Carlo (MC) methods only need "experience (sample sequences of states, actions, and rewards from actual (or simulated) interaction with an environment." Learning from actual experience is a big deal because then no knowledge of the mechanics and dynamics of the environment is needed to learn optimal behaviors. When we learn from simulated experience, this is also very useful becauase although we need a model of the environment, we only need to use the model to generate transition probabilities for samples, not for every possible transition like dynamic programming requires. 

"Monte Carlo methods enable us to solve the RL problem by averaging sample returns." We'll only look at Monte Carlo methods for episodic tasks so that we can be certain that we're only dealing with well-defined returns. Policies and value estimates are only changed at the end of an episode. 

MC samples and averages future returns over state-action pairs. This is similar to the k-armed bandit methods from Chapter 2; the bandit methods sampled and averaged reward for each action. The big difference now is that we have more than one state, and that we allow the states to interact with each other. Recall, this is the full RL problem alluded to at the end of Chapter 2 and covered throughout Chapter 3. "The return after taking an action in one state depends on the actions taken in later states in the same episode." This problem becomes nonstationary because we are continuously learning to make different action choices. 

We adapt the idea of Generalized Policy Iteration (GPI) to handle the nonstationarity of the problem; we use samples from the MDP to learn the value function. This is in contrast to when dynamic programming was used to directly compute the value function using GPI. Each piece of GPI is extended from dynamic programming to MC, where here we use sample experience to learn the policy $\pi$ and $v_\pi$ and $q_\pi$.

### 5.1: Monte Carlo Prediction

We will use the Monte Carlo method to learn the state-action value function for a policy. The policy will be given. To estimate the state-action values from experience, we'll average the observed returns after each time we are in a state. This average should converge to the expected value for each state. Imagine we want to estimate $v_\pi(s)$ given a set of episodes that we've gathered by following $\pi$ and passing through states $s$. Each time we see state $s$ in an episode is called a visit to $s$. $s$ may be visited multiple times in an episode. The first time we see $s$ in an episode will be called the first visit to $s$. First visit Monte Carlo method works to estimate $v_\pi(s)$ as every return after the first visit to $s$. Every visit Monte Carlo averages the returns following all visits to $s$. 

First visit and every visit Monte Carlo both converge to $v_\pi(s)$ as the number of visits goes to infinity. In the case of first visit Monte Carlo this is easy to see because each return is an i.i.d estimation of $v_\pi(s)$. By the law of large numbers the sequence of averages converges to the expected value. Every visit Monte Carlo isn't as simple but its estimates of $v_\pi(s)$ also converge quadratically. 


### 5.2: Monte Carlo Estimation of Action Values

When we don't have a model of the environment, then its useful to estimate action values rather than state values. When we have a model, state values by themselves are enough for us to form a policy. We just look one state ahead and choose the one with highest value. However, when we don't have a model, state values by themselves aren't enough. Need to directly estimate the value of each action for the values to be useful and direct a policy. So one of main goals with Monte Carlo methods is to estimate $q_*$. To do this, we'll look at the policy evaluation problem for action values.

When doing policy evaluation for action-values we want to estimate $q_\pi(s, a)$. The expected return when starting from a state $s$, taking an action $a$ and afterwards following policy $\pi$. "The Monte Carlo methods for this are essentially the same as the state values, except now we talk about visits to a state-action pair rather than to a state." Both Every-visit and First-visit MC still converge quadratically to the true expected reward values as the number of visits to each state-action pair goes to infinity. 

Only problem is that lots of state-action pairs will never be visited. If we're following a deterministic policy then MC will only ever observe returns for one action from that state. Since there will be no returns for the other actions, MC will never learn to estimate the returns of those actions. It is necessary to estimate returns for all the actions available in a state, not only the actions the policy prefers.

This is the problem of maintaining exploration. For the policy evaluation to still work for action values, we have to ensure that exploration continues. We can do this by saying that each episode starts in a state-action pair and give each pair a probability greater than zero of being selected. This is called exploring starts and it guarantees that each pair will be visited an infinite number of times as the number of episodes goes to infinity. Assuming exploring starts isn't always reliable. We cannot depend on it when learning from real interaction. The most common alternative is to only use policies that are stochastic and have nonzero probability for each action in a state. 

### 5.3: Monte Carlo Control

Monte Carlo estimation can be used to approximate an optimal policy. We want to follow the same idea outlined in the dynamic programming chapter, using generalized policy iteration. In generalized policy iteration, we maintain an approximate value function and an approximate policy. The value function is iteratively updated to look like the value function for the current policy, and the policy is iteratively improved compared to the current value function. These two processes together push both the policy and the value function towards being optimal. For starters, we consider Monte Carlo version of classical policy iteration. We do alternating steps of policy iteration and policy improvement. Start with an arbitrary policy $\pi_0$ and end with optimal policy $\pi_*$ and optimal action-value function $q_*$

$\pi_0 \stackrel{E}{\rightarrow} q_{\pi_0} \stackrel{I}{\rightarrow} \pi_1 \stackrel{E}{\rightarrow} q_{\pi_1} \stackrel{I}{\rightarrow} \pi_2 \stackrel{E}{\rightarrow} \dotsb \stackrel{I}{\rightarrow} \pi_* \stackrel{E}{\rightarrow} q_*$

E shows a policy evaluation and I shows a policy improvement. Assume we do explore an infinite number of episodes and that the episodes are started with exploring starts. With this assumption the MC methods will compute each $q_{\pi_k}$ precisely, for any $\pi_k$. 

In the case of this policy improvement, we have an action-value function and so don't need a model of the environment to construct a policy. For any action-value function $q$, the greedy policy is the policy that, for each state, chooses an action with the highest action-value deterministically. 

$\pi(s) \stackrel{.}{=} argmax_a(q(s,a))$

Policy improvement can be performed by building the $\pi_{k+1}$ as the greedy policy for the current action-value function. The policy evaluation theorem from section 4.2 then applies here to $\pi_k$ and to $\pi_{k+1}$ because for all states in the state-space,

$q_{\pi_k} = q_{\pi_k} (s, argmax_a(q(s,a)))$

$\quad = max_a q_{\pi_k} (s,a)$

$\quad \geq q_{\pi_k} (s, \pi_k(s))$

$\quad \geq v_{\pi_k}(s)$

To reiterate from earlier discussion of this theorem, it guarantees that $\pi_{k+1}$ is more optimal than $\pi_k$ until an optimal policy is reached. This guarantees that the process overall converges to an optimal policy and value function. 

We have two assumptions we made that we need to remove. We assume that the episodes had exploring starts and that we have an infinite number of episodes for policy evaluation to operate on. For now we will only focus on the infinite number of episodes assumption. We can solve this problem by holding firm to the idea that we approximate $q_{\pi_k}$ in each iteration. The second option is to give up trying to complete policy evaluation before going back to policy improvement. During each evaluation step we move the approximation towards $q_{\pi_k}$, but don't expect to actually get close except for over many many steps. Extreme forms of this idea include value iteration, where only one step of policy evaluation is performed between steps of policy improvement. There is an inplace version of value iteration that is even more extreme, it alternates between improvement and evaluation steps for single states.

It is pretty natural for Monte Carlo policy iteration to alternate between evaluation and improvement episode-by-episode. After each episode, returns are used for policy evaluation, and then the policy is improved at all of the states visited in the episode. 

Monte Carlo with Exploring starts is along these lines and is outlined below.

Monte Carlo ES (Exploring Starts) for approximating $\pi \approxeq \pi_*$

Initialize:  
$\quad \pi(s) \in A(s) \quad$ arbitrarily, $\forall \space s \in S$  
$\quad Q(s, a) \in \mathbb{R}$ arbitrarily, $\forall \space s \in S, a \in A(s)$  
$\quad Returns(s, a) \leftarrow$ empty list, $\forall \space s \in S, a \in A(s)$  

Loop forever:  
$\quad$ Choose $S_0 \in S, A_0 \in A(S_0)$ randomly such that all pairs have probability > 0  
$\quad$ Generate an episode from $S_0, A_0$, following $\pi$: $\S_0, A_0, R_1, \dots, S_{T-1}, A_{T-1}, R_T$  
$\quad G \leftarrow 0$  
$\quad$ Loop through each step of episode:  
$\quad \quad G \leftarrow \gamma G + R_{t+1}$  
$\quad \quad$Unless the pair $S_t, A_t$ appears in the episode:  
$\quad \quad \quad$Append G to $Returns(s, a)$  
$\quad \quad \quad Q(S_t, A_t) \leftarrow average(Returns(S_t,A_t))$  
$\quad \quad \quad \pi(S_t) \leftarrow argmax_a Q(S_t, a)$  